In [1]:
!pip install transformers
!pip install datasets


     |████████████████████████████████| 4.0 MB 7.6 MB/s 
     |████████████████████████████████| 77 kB 5.8 MB/s 
     |████████████████████████████████| 6.6 MB 45.9 MB/s 
     |████████████████████████████████| 895 kB 62.5 MB/s 
     |████████████████████████████████| 596 kB 43.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 8.6 MB/s 
     |████████████████████████████████| 1.1 MB 36.4 MB/s 
     |████████████████████████████████| 136 kB 34.1 MB/s 
     |████████████████████████████████| 212 kB 45.7 MB/s 
     |████████████████████████████████| 127 kB 51.7 MB/s 
     |████████████████████████████████| 271 kB 52.7 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 144 kB 67.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import datasets
from datasets import Dataset
from transformers import AutoModelForSequenceClassification
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer


In [4]:
import os
import json
import pandas as pd
BASE_DIR = "/content/drive/MyDrive/NLP_ENSAE/"
json_name = "Digital_Music.json"
os.chdir(BASE_DIR)
df = pd.read_json(os.path.join(BASE_DIR, json_name), lines = True)
df = df.drop(['image', 'reviewerID', 'asin'], axis = 1)
df.dropna(inplace=True)
df.reset_index(inplace=True)

In [5]:
# def get_dataset_format(row_df):
#     return {'label' : row_df['overall'] , 'text' : row_df['reviewText'] }

# df['dataset_format'] = df.apply(get_dataset_format, axis = 1)

In [6]:
train_df = pd.DataFrame({
     
     "label" : df[:int(0.8*df.shape[0])]['overall'].tolist(),
     "text" : df[:int(0.8*df.shape[0])]['reviewText'].tolist()
})

test_df = pd.DataFrame({
     "label" : df[int(0.8*df.shape[0]):]['overall'].tolist(),
     "text" : df[int(0.8*df.shape[0]):]['reviewText'].tolist()
})

In [7]:
# dataset_ = df['dataset_format']
# dataset_ = pd.DataFrame(dataset_, columns=['dataset_format'])
# #shuffle dataset
# dataset_ = dataset_.sample(frac = 1)
# train_df = dataset_[:int(0.8*dataset_.shape[0])]
# test_df = dataset_[int(0.8*dataset_.shape[0]):]

In [8]:
train_df.head()

,label,text
0,5,Keith Green / So you wanna go back to Egypt......
1,5,"Clever,inspired and moving. This is a great al..."
2,5,Keith Green is a bit of legend in some Christi...
3,5,Keith's music is a timeless message. Since hi...
4,5,"Don Francisco's ""Early Works"" are filled with ..."


In [9]:
from datasets import Dataset


In [34]:
train_dataset = Dataset.from_pandas(train_df).select(range(1000))
test_dataset = Dataset.from_pandas(test_df).select(range(1000))

my_dataset_dict = datasets.DatasetDict({"train":train_dataset,"test":test_dataset})


In [35]:
type(my_dataset_dict)

datasets.dataset_dict.DatasetDict

In [36]:
my_dataset_dict.keys()

dict_keys(['train', 'test'])

In [37]:
dataset = my_dataset_dict

In [38]:
dataset['train'][10]

{'label': 5,
 'text': 'This was the first album I ever listened to by JMT, so I have always thought of him as quiet, meditative, and worshipful. How much that is the case, I don\'t know, because I have not branched out to many of his other works since. I find this double CD to contain just about all I need. Our family has found a few of these songs a real blessing to learn to sing together quietly. The album describes itself well: a great collection of songs that will take you to the "quiet side". Sort of like experiencing "the Lord . . . makes me lie down in green pastures, he leads me beside quiet waters, he restores my soul". If you are not a Catholic, you may find the topical arrangement of the songs and some of the inner artwork oriented this way. Even though I normally stay far from most things Catholic, I wouldn\'t pass this one up. It is universal. If you like the "quiet side" and songs with a bit more Scripture and depth than commonly found, I recommend you don\'t pass it up e

In [39]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True, batch_size = 8)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/resolve/main/voc

  0%|          | 0/125 [00:00<?, ?ba/s]

  0%|          | 0/125 [00:00<?, ?ba/s]

In [40]:
small_train_dataset = tokenized_datasets["train"]
small_eval_dataset = tokenized_datasets["test"]

In [41]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-cased", num_labels=6)

loading configuration file https://huggingface.co/distilbert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ebe1ea24d11aa664488b8de5b21e33989008ca78f207d4e30ec6350b693f073f.302bfd1b5e031cc1b17796e0b6e5b242ba2045d31d00f97589e12b458ebff27a
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-cased",
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_"

In [42]:
#sum(p.numel() for p in model.parameters()) #Bert based case

In [43]:
sum(p.numel() for p in model.parameters())


65786118

In [44]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer", do_train = True, 
                                  num_train_epochs = 25,
                                  per_device_train_batch_size = 8, )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [28]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [29]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [45]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", do_train = True,
                                  num_train_epochs = 25,
                                  per_device_train_batch_size = 8,)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [46]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [47]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_type_ids, text. If token_type_ids, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1000
  Num Epochs = 25
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3125


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.873722,0.747000
2,No log,0.862556,0.751000
3,No log,0.767710,0.737000
4,0.639200,0.930513,0.729000


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_type_ids, text. If token_type_ids, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_type_ids, text. If token_type_ids, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_type_ids, text. If token_type_ids, text are not expected by `DistilBertForSequenceClassificatio

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.873722,0.747000
2,No log,0.862556,0.751000
3,No log,0.767710,0.737000
4,0.639200,0.930513,0.729000
5,0.639200,1.125713,0.726000
6,0.639200,1.240214,0.708000
7,0.639200,1.455185,0.741000
8,0.171700,1.670248,0.765000
9,0.171700,1.674488,0.736000
10,0.171700,1.740924,0.732000


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_type_ids, text. If token_type_ids, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_type_ids, text. If token_type_ids, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_type_ids, text. If token_type_ids, text are not expected by `DistilBertForSequenceClassificatio

TrainOutput(global_step=3125, training_loss=0.13200035843223334, metrics={'train_runtime': 1117.7455, 'train_samples_per_second': 22.366, 'train_steps_per_second': 2.796, 'total_flos': 3311921203200000.0, 'train_loss': 0.13200035843223334, 'epoch': 25.0})